# QA1. Changing associations

In [2]:
from koselleck import *
words=get_valid_words()

In [3]:
dfmodels = get_pathdf_models().query('period_len==5 & 1720<=period_start<1900')
dfmodels

,corpus,period_start,period_end,path,path_vocab,run,period,period_len
506,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_01,1720-1725,5
513,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_02,1720-1725,5
495,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_03,1720-1725,5
504,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_04,1720-1725,5
508,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_05,1720-1725,5
...,...,...,...,...,...,...,...,...
1656,bpo,1895,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_21,1895-1900,5
1665,bpo,1895,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_22,1895-1900,5
1663,bpo,1895,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_23,1895-1900,5
1648,bpo,1895,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_24,1895-1900,5


In [65]:
def do_gen_localdist(pathdf,words=None,k=100,force=False):
    mpath=pathdf.iloc[0].path
    ofnfn=os.path.join(os.path.dirname(mpath), 'dists.pkl')
    res=pd.DataFrame([{'path_dists':ofnfn}])
    if not force and os.path.exists(ofnfn): return res
    
    if not os.path.exists(mpath): return pd.DataFrame()
    m=load_model(mpath)

    if not words: words=get_valid_words()
    words=list(set(words) & set(m.wv.key_to_index.keys()))
    
#     vecs=m.wv.get_normed_vectors()
    vecs=m.wv.vectors
    dfmat = pd.DataFrame([
        vecs[m.wv.key_to_index[w]]
        for w in words
    ],index=words)
    
    mat_cos = fastdist.cosine_pairwise_distance(dfmat.values,return_matrix=True)
    dfsims = pd.DataFrame(mat_cos, index=words, columns=words)
    dfdist=2-dfsims
    dfdist=round(dfdist,8)
    dfdist.to_pickle(ofnfn)
    return res

In [66]:
dfdist = do_gen_localdist(dfmodels.iloc[:1], force=True)
dfdist

,path_dists
0,/home/ryan/github/koselleck/data/models/bpo/17...


In [67]:
# pd.read_pickle('/home/ryan/github/koselleck/data/models/bpo/1720-1725/run_01/neighbs.pkl')

In [73]:
def gen_localdists(dfmodels,words=None,k=100,num_proc=1):
    if not words: words=get_valid_words()
    return pmap_groups(
        do_gen_localdist,
        dfmodels.groupby(['corpus','period','run']),
        num_proc=num_proc,
        kwargs=dict(
            k=k,
            words=words
        )
    )

In [74]:
odf=gen_localdists(dfmodels, num_proc=7)

Mapping do_gen_localdist [x7]: 100%|██████████| 900/900 [36:57<00:00,  2.46s/it]


In [75]:
odf

path_dists
corpus period    run                                                      
bpo    1720-1725 run_01  /home/ryan/github/koselleck/data/models/bpo/17...
                 run_02  /home/ryan/github/koselleck/data/models/bpo/17...
                 run_03  /home/ryan/github/koselleck/data/models/bpo/17...
                 run_04  /home/ryan/github/koselleck/data/models/bpo/17...
                 run_05  /home/ryan/github/koselleck/data/models/bpo/17...
...                                                                    ...
       1895-1900 run_21  /home/ryan/github/koselleck/data/models/bpo/18...
                 run_22  /home/ryan/github/koselleck/data/models/bpo/18...
                 run_23  /home/ryan/github/koselleck/data/models/bpo/18...
                 run_24  /home/ryan/github/koselleck/data/models/bpo/18...
                 run_25  /home/ryan/github/koselleck/data/models/bpo/18...

[900 rows x 1 columns]

In [76]:
pd.read_pickle(odf.path_dists[0]).sort_values('history')

,step,ham,rust,sash,epistle,ide,countess,president,nest,vengeance,...,abridgment,bigotry,flu,inscription,amp,colt,date,rag,ad,tau
history,2.014369,2.019049,1.821995,2.209470,1.386828,2.078806,1.820859,1.657013,2.074100,1.991037,...,0.956542,1.712787,1.840902,1.527292,1.905999,2.243127,1.560184,1.746198,1.969952,1.664787
divinity,2.029598,1.832435,1.827352,1.759680,1.781338,1.934977,1.889985,1.468768,1.835999,1.914943,...,1.015032,1.550306,1.866117,1.538077,1.697976,1.912719,2.044833,1.795132,1.936137,1.643838
treatise,2.046525,1.635215,1.659913,1.989571,1.337470,2.051283,2.008618,1.820566,2.151630,1.821556,...,0.929122,1.700021,2.063867,1.524589,1.813595,2.333583,1.928864,1.651126,1.759326,1.650673
poetry,2.126051,1.931445,1.668230,1.691620,1.592269,1.828797,1.725812,1.696517,1.969483,1.600446,...,1.235370,1.641758,1.813968,1.877767,1.593891,1.961582,2.106893,1.507793,2.027110,1.632520
usefulness,1.946113,1.750102,1.564633,1.866428,1.559641,2.034568,1.655938,1.781961,2.037778,1.765480,...,1.043296,1.572308,1.702422,1.615984,1.755639,2.008413,1.766832,1.446988,2.087858,1.750800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
caw,1.970062,1.485991,1.905453,1.876854,2.150718,1.247379,2.492706,2.652680,2.162526,2.235711,...,2.017182,2.146748,1.397383,2.136595,1.456847,1.318111,1.981006,1.572042,1.462781,1.591632
poll,1.793651,2.176339,2.054645,2.192672,2.223621,2.397179,1.817775,1.689463,1.739955,1.869673,...,2.367812,2.364058,1.862496,1.857905,2.169539,2.280605,1.847910,2.568403,2.028552,2.079920
sire,2.360774,1.539033,2.156727,1.578919,2.197012,1.719393,1.433890,2.074600,1.790105,1.834312,...,2.125527,2.034995,1.823732,1.874347,1.401670,1.716666,2.377726,1.593525,1.979719,1.862682
toe,1.992496,1.406788,2.062327,1.335449,2.215589,1.428893,2.000094,2.419052,1.961719,1.935470,...,2.167691,2.105979,1.545468,2.077368,1.341244,1.447999,2.325151,1.451102,1.317310,1.421149
